In [1]:
import pandas as pd
from tqdm import tqdm

# settings
locations = {
    'Zch_Stampfenbachstrasse': {
        'lat': 47.382390,
        'lon': 8.541740,
    }, 
    'Zch_Schimmelstrasse': {
        'lat': 47.370270,
        'lon': 8.524890,
    },
    'Zch_Heubeeribüel': {
        'lat': 47.387810,
        'lon': 8.527100,
    },
    'Zch_Rosengartenstrasse': {
        'lat': 47.395100,
        'lon': 8.525920,
    },
}

In [2]:
# check if data is already downloaded
try:
    data = pd.read_parquet('data.parquet')

# if not, download data
except FileNotFoundError:
    # Download data from 1983 to 2021
    data = None
    for year in tqdm(range(1983, 2022)):
        url = f'https://data.stadt-zuerich.ch/dataset/ugz_luftschadstoffmessung_stundenwerte/download/ugz_ogd_air_h1_{year}.csv'
        temp = pd.read_csv(url, sep=',')
        data = temp if data is None else pd.concat([data, temp])

    # save data to parquet file
    data.to_parquet('data.parquet')

In [3]:
# Datum to datetime
data['Datum'] = pd.to_datetime(data['Datum'], format='%Y-%m-%dT%H:%M%z')
data = data.sort_values(by=['Datum'])

# Remove timezone
data['Datum'] = data['Datum'].dt.tz_localize(None)

# Replace Standort with lat and lon columns
data['Lat'] = data['Standort'].apply(lambda x: locations[x]['lat'])
data['Lon'] = data['Standort'].apply(lambda x: locations[x]['lon'])

# Only use Data where "Status" is "bereinigt"
data = data[data['Status'] == 'bereinigt']
data = data.drop(columns=['Status'])

# Drop unnecessary "Intervall" column
data = data.drop(columns=['Intervall'])

# Combine "Parameter" and "Einheit" columns
data['Parameter'] = data['Parameter'] + ' in ' + data['Einheit']
data = data.drop(columns=['Einheit'])

In [4]:
# export cleaned data to parquet file
data.to_parquet('data_cleaned.parquet')